In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
conda install python=3.7

In [ ]:
conda install -c conda-forge gdcm -y

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from os import listdir, mkdir
import matplotlib.pyplot as plt
from glob import glob 
from zipfile import ZipFile
from keras.models import Sequential
import keras
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.python.keras.layers import Dense,Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import pydicom
import gdcm

In [ ]:
basepath = "../input/rsna-str-pulmonary-embolism-detection/"
train = pd.read_csv(basepath+"train.csv")
test = pd.read_csv(basepath+"test.csv")
train.head()

In [ ]:
train = train.drop(['qa_motion','qa_contrast','flow_artifact','true_filling_defect_not_pe'] , 1)
train.columns

In [ ]:
train.head()

In [ ]:
#Building CNN model:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256,256,3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
#Lets tell the model what cost and optimization method to use and fit the model to our training data set
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train data:

In [ ]:
import os
PathDicom_train = basepath+"train/"
lstFilesDCM_train = []  # create an empty list
count = 0
for dirName, subdirList, fileList in os.walk(PathDicom_train):
    for filename in fileList:
        if ".dcm" in filename.lower() and count<5000:  # check whether the file's DICOM
            lstFilesDCM_train.append(os.path.join(dirName,filename))
        else:
            break
        count+=1

In [ ]:
# Get ref file
RefDs = pydicom.read_file(lstFilesDCM_train[0])

# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM_train))

# Load spacing values (in mm)
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

In [ ]:
x = np.arange(0.0, (ConstPixelDims[0]+1)*ConstPixelSpacing[0], ConstPixelSpacing[0])
y = np.arange(0.0, (ConstPixelDims[1]+1)*ConstPixelSpacing[1], ConstPixelSpacing[1])
z = np.arange(0.0, (ConstPixelDims[2]+1)*ConstPixelSpacing[2], ConstPixelSpacing[2])


In [ ]:
#os.chdir('patil.shru/')
# The array is sized based on 'ConstPixelDims'
ArrayDicom_train = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

# loop through all the DICOM files
for filenameDCM in lstFilesDCM_train:
    # read the file
    ds = pydicom.read_file(filenameDCM)
    # store the raw image data
    ArrayDicom_train[:, :, lstFilesDCM_train.index(filenameDCM)] = ds.pixel_array
    ArrayDicom_train=ArrayDicom_train.astype('float32').tobytes()

Test data:

In [ ]:
PathDicom_test = basepath+"test"
lstFilesDCM_test = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicom_test):
    for filename in fileList:
        if ".dcm" in filename.lower() and count<10000:  # check whether the file's DICOM
            lstFilesDCM_test.append(os.path.join(dirName,filename))
        else:
            break
        count+=1


In [ ]:
# Get ref file
RefDs = pydicom.read_file(lstFilesDCM_test[0])

# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM_test))

# Load spacing values (in mm)
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

In [ ]:
# The array is sized based on 'ConstPixelDims'
ArrayDicom_test = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

# loop through all the DICOM files
for filenameDCM in lstFilesDCM_test:
    # read the file
    ds = pydicom.read_file(filenameDCM)
    # store the raw image data
    ArrayDicom_test[:, :, lstFilesDCM_test.index(filenameDCM)] = ds.pixel_array 

In [ ]:
#from keras.preprocessing.image import ImageDataGenerator
#ImageDataGenerator helps in Image Augmentation. IMAGE_SIZE will be used in VGG, ResNet implementation to resize image
#s=32
#IMAGE_SIZE = [224, 224]
#set validation split
#trainGen=ImageDataGenerator(rescale=1./255,shear_range= 0.2,zoom_range= 0.2,horizontal_flip= True,rotation_range= 40,width_shift_range = 0.2,height_shift_range = 0.2)

#testGen = ImageDataGenerator(rescale=1./255)

#train_imgs = trainGen.flow_from_directory('ArrayDicom_train', class_mode='categorical', batch_size=32)
#test_imgs = testGen.flow_from_directory('lstFilesDCM_test', class_mode='binary', batch_size=32)

In [ ]:
train_sub = train.head(512)
train_sub.head()

In [ ]:
model.fit(ArrayDicom_train, train_sub, batch_size = 32, epochs = 100)
model.save('model.h1')